# Replicating CIFAR-10 Analysis with ResNet-20

In [1]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim

>Check Cuda

In [2]:
print(torch.__version__)
# Check CUDA version supported by PyTorch
print("CUDA version supported by PyTorch:", torch.version.cuda)

2.4.1+cu124
CUDA version supported by PyTorch: 12.4


In [3]:
def check_cuda():
    if torch.cuda.is_available():
        print(
            "CUDA is available. You have", torch.cuda.device_count(), "CUDA device(s)."
        )
        for i in range(torch.cuda.device_count()):
            print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("CUDA is not available.")


check_cuda()

CUDA is available. You have 1 CUDA device(s).
Device 0: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [4]:
# define the SimpleNN mode;
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.silu = nn.SiLU()

    def forward(self, x):
        out = self.silu(self.bn1(self.conv1(x)))
        out = F.max_pool2d(out, 2)
        out = self.silu(self.bn2(self.conv2(out)))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.silu(self.fc1(out))
        out = self.silu(self.fc2(out))
        out = self.fc3(out)
        return out

> pre-processing from simpleNN


In [6]:
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

transform_val = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean, std)]
)

>Dataloader

In [7]:
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

train_set = CIFAR10(
    root=DATA_ROOT, mode="train", download=True, transform=transform_train
)
val_set = CIFAR10(root=DATA_ROOT, mode="val", download=True, transform=transform_val)

# construct dataloader
train_loader = DataLoader(
    train_set,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)
val_loader = DataLoader(
    val_set,
    batch_size=VAL_BATCH_SIZE,
    shuffle=False,
    num_workers=4,
)

Using downloaded and verified file: ./data\cifar10_trainval_F22.zip
Extracting ./data\cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data\cifar10_trainval_F22.zip
Extracting ./data\cifar10_trainval_F22.zip to ./data
Files already downloaded and verified


In [11]:
INITIAL_LR = 0.1

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 1e-4

In [12]:
def train_val(
    optimizer,
    criterion,
    model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    INITIAL_LR=INITIAL_LR,
    REG=REG,
):
    # some hyperparameters
    # total number of training epochs

    EPOCHS = 30

    # the folder where the trained model is saved

    CHECKPOINT_FOLDER = "./saved_model"

    # start the training/validation process

    # the process should take about 5 minutes on a GTX 1070-Ti
    # if the code is written efficiently.
    best_val_acc = 0

    current_learning_rate = INITIAL_LR

    print("==> Training starts!")
    print("=" * 50)

    for i in range(0, EPOCHS):
        #######################

        # your code here
        # switch to train mode
        model.train()
        #######################

        print("Epoch %d:" % i)
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0  # track training loss if you want

        # Train the model for 1 epoch.

        for batch_idx, (inputs, targets) in enumerate(train_loader):
            ####################################

            # your code here
            # copy inputs to device
            inputs, targets = inputs.to(device), targets.to(device)
            # compute the output and loss

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # zero the gradient

            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            train_loss += loss.item()

            # count the number of correctly predicted samples in the current batch
            total_examples += targets.size(0)
            _, predicted = outputs.max(1)
            correct_examples += predicted.eq(targets).sum().item()
            ####################################

        print(
            f"CUDA memory allocated: {torch.cuda.memory_allocated(device) / 1e6:.2f} MB"
        )
        avg_loss = train_loss / len(train_loader)
        avg_acc = correct_examples / total_examples
        print("Training loss: %.4f, Training accuracy: %.4f" % (avg_loss, avg_acc))

        # Validate on the validation dataset
        #######################

        # your code here
        # switch to eval mode
        model.eval()

        #######################

        # this help you compute the validation accuracy
        total_examples = 0
        correct_examples = 0

        val_loss = 0  # again, track the validation loss if you want

        # disable gradient during validation, which can save GPU memory

        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                ####################################

                # your code here
                # copy inputs to device
                inputs, targets = inputs.to(device), targets.to(device)
                # compute the output and loss

                outputs = model(inputs)
                loss = criterion(outputs, targets)

                # count the number of correctly predicted samples in the current batch

                total_examples += targets.size(0)
                _, predicted = outputs.max(1)
                correct_examples += predicted.eq(targets).sum().item()
                val_loss += loss.item()
                ####################################

        avg_loss = val_loss / len(val_loader)
        avg_acc = correct_examples / total_examples

        print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))

        # save the model checkpoint

        if avg_acc > best_val_acc:
            best_val_acc = avg_acc

            if not os.path.exists(CHECKPOINT_FOLDER):
                os.makedirs(CHECKPOINT_FOLDER)
            print("Saving ...")

            state = {
                "state_dict": model.state_dict(),
                "epoch": i,
                "lr": current_learning_rate,
            }
            checkpoint_path = os.path.join(
                CHECKPOINT_FOLDER, f"LR{INITIAL_LR}_REG{REG}.pth"
            )
            torch.save(state, checkpoint_path)

        print("")

    print("=" * 50)

    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

In [13]:
for lr in LR_LIST:
    model = SimpleNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        model.parameters(), lr=lr, momentum=MOMENTUM, weight_decay=REG
    )
    print(f"Training with learning rate: {lr}")
    train_val(optimizer, criterion, model, INITIAL_LR=lr)
for reg in REG_LIST:
    model = SimpleNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=reg
    )
    print(f"Training with regularization strength: {reg}")
    train_val(optimizer, criterion, model, REG=reg)

Training with learning rate: 1.0
==> Training starts!
Epoch 0:
CUDA memory allocated: 18.93 MB
Training loss: nan, Training accuracy: 0.1036
Validation loss: nan, Validation accuracy: 0.1028
Saving ...

Epoch 1:
CUDA memory allocated: 18.93 MB
Training loss: nan, Training accuracy: 0.0999
Validation loss: nan, Validation accuracy: 0.1028

Epoch 2:
CUDA memory allocated: 18.93 MB
Training loss: nan, Training accuracy: 0.0999
Validation loss: nan, Validation accuracy: 0.1028

Epoch 3:
CUDA memory allocated: 18.93 MB
Training loss: nan, Training accuracy: 0.0999
Validation loss: nan, Validation accuracy: 0.1028

Epoch 4:
CUDA memory allocated: 18.93 MB
Training loss: nan, Training accuracy: 0.0999
Validation loss: nan, Validation accuracy: 0.1028

Epoch 5:
CUDA memory allocated: 18.93 MB
Training loss: nan, Training accuracy: 0.0999
Validation loss: nan, Validation accuracy: 0.1028

Epoch 6:
CUDA memory allocated: 18.93 MB
Training loss: nan, Training accuracy: 0.0999
Validation loss: nan

# Bonus: with learning rate decay

The following code can help you adjust the learning rate during training. You need to figure out how to incorporate this code into your training loop.
```python
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
```